In [73]:
library(ggtranscript)
library(dplyr)
library(readr)
library(ggplot2)
library(grid)

In [44]:
mytheme <- theme_minimal() + theme(axis.text = element_text(color = "black"),
                                 axis.ticks = element_line(linewidth = 0.2), 
                                 axis.title = element_text(size=7, vjust = -0.5),
                                 legend.title = element_text(size = 7, face = "bold"),
                                 legend.margin = margin(r = 0, l = 0, t = 0, b = 0),
                                 legend.box.margin = margin(-10, 3, -10, -7),
                                 legend.key.size = unit(0.2, "cm"),
                                 panel.border = element_rect(linewidth = 0.4, fill = FALSE), 
                                 panel.background = element_blank(),   
                                 panel.grid = element_line(linewidth =0.2),
                                 plot.margin = margin(t = 10, r = 10, b = 10, l = 10),
                                 plot.title = element_text(face="bold", hjust=0.5),
                                 strip.text = element_text(size=7, face="bold"),   
                                 strip.background = element_blank(),
                                 text = element_text(family = "Helvetica",color="black", size=7))  

In [3]:
gtf_path = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/poder_v1_cds.gtf'
gtf = rtracklayer::import(gtf_path)
gtf = gtf %>% dplyr::as_tibble()

In [16]:
ref_gtf_path = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/ref/annot.gtf'
ref_gtf = rtracklayer::import(ref_gtf_path)
ref_gtf = ref_gtf %>% dplyr::as_tibble()

In [55]:
colnames(ref_gtf)

[1] "seqnames"                 "start"                   
 [3] "end"                      "width"                   
 [5] "strand"                   "source"                  
 [7] "type"                     "score"                   
 [9] "phase"                    "gene_id"                 
[11] "gene_type"                "gene_name"               
[13] "level"                    "tag"                     
[15] "transcript_id"            "transcript_type"         
[17] "transcript_name"          "exon_number"             
[19] "exon_id"                  "transcript_support_level"
[21] "havana_transcript"        "hgnc_id"                 
[23] "havana_gene"              "ont"                     
[25] "protein_id"               "ccdsid"                  
[27] "artif_dupl"

In [11]:
curr_gid = 'ENSG00000234745.14'
nov_tids = c('transcript_345653')

In [89]:
known_tids <- ref_gtf %>%
        subset(gene_id == curr_gid) %>%
        pull(transcript_id)
gname <- ref_gtf %>%
        subset(gene_id == curr_gid) %>%
        pull(gene_name)

# plot novel transcript exons and cdss
temp_gtf <- gtf %>%
  subset(transcript_id %in% nov_tids & type == 'exon')
temp_gtf = temp_gtf %>% mutate(annot_source = 'PODER')

temp_cds_gtf <- gtf %>%
  subset(transcript_id %in% nov_tids & type == 'CDS')
temp_cds_gtf = temp_cds_gtf %>% mutate(annot_source = 'PODER')

# plot known transcript exons and cdss
temp_ref_gtf <- ref_gtf %>%
  subset(transcript_id %in% known_tids & type == 'exon')
temp_ref_gtf = temp_ref_gtf %>% mutate(annot_source = 'GENCODE')

temp_ref_cds_gtf <- ref_gtf %>%
  subset(transcript_id %in% known_tids & type == 'CDS')
temp_ref_cds_gtf = temp_ref_cds_gtf %>% mutate(annot_source = 'GENCODE')


temp = bind_rows(temp_gtf, temp_ref_gtf)
temp_cds = bind_rows(temp_cds_gtf, temp_ref_cds_gtf)

c_dict = c('PODER'='#a2ae59', 'GENCODE'='#816755')

p = temp %>%
    ggplot(aes(
        xstart = start,
        xend = end,
        y = transcript_id
    )) +
    geom_range(
        aes(fill=annot_source),
        height = 0.25,
        linewidth=0
    )+
    geom_range(
        data = temp_cds,
        aes(fill=annot_source),
        linewidth=0
    ) +
    geom_intron(
        data = to_intron(temp, "transcript_id"),
        aes(strand = strand),
        arrow.min.intron.length = 500,
        linewidth =0.25,
        arrow=arrow(length=unit(0.025, "inches"))
    )+
        scale_fill_manual(values = c_dict)+
    labs(fill="", y="")+#, title=gname)+
    mytheme+
    theme(legend.position = "none")

h = 1.8658
w = 3.6389

file_name = 'hlab_example.pdf'
pdf(file_name, width=w, height=h)
print(p)
dev.off()

pdf 
  2

In [48]:
# mytheme